Yiting Wang 4646909520

(b)

In [1]:
import warnings
warnings.filterwarnings("ignore")

In [34]:
import os
import numpy as np
directory = '../data/Book Files/books/'

c(i)

In [15]:
def create_corpus(directory):
    corpus = []
    for root, _, files in os.walk(directory):
        for f in files:
            with open(root + f, encoding='ascii', errors='ignore') as book:
                book_corpus = book.read().lower()
                corpus.append(book_corpus)
            print('Text: {}, String length: {}'.format(f, len(corpus[-1])))
        corpus = sorted(corpus, key=lambda x : len(x))
    return corpus

In [25]:
Russell_corpus = create_corpus(directory)
print([len(book) for book in Russell_corpus])

Text: AIIMAT.txt, String length: 746219
Text: MLOE.txt, String length: 412226
Text: OKEWFSMP.txt, String length: 405741
Text: TAM.txt, String length: 514652
Text: TAMatter.txt, String length: 766542
Text: THWP.txt, String length: 2005566
Text: TPP.txt, String length: 244306
[244306, 405741, 412226, 514652, 746219, 766542, 2005566]


C(ii)

In [27]:
chars = []
for n in Russell_corpus:
    chars.extend(list(set(set(n))))

chars = sorted(list(set(chars)))
char_dict = dict((c, i) for i, c in enumerate(chars))
int_dict = dict((i, c) for i, c in enumerate(chars))

c(iii), c(iv)

In [37]:
def window_Corpus(corpus, win_size):
    input_list, output_list = [], []
    for w in range(0, len(corpus) - win_size + 1, 1):
        seq_in = corpus[w :w + win_size - 1]
        seq_out = corpus[w + win_size - 1]
        input_list.append([char_dict[c] for c in seq_in])
        output_list.append(char_dict[seq_out])
    return input_list, output_list

In [41]:
win_size = 100

for n in Russell_corpus:
    input_list, output_list = [], []
    temp_in, temp_out = window_Corpus(n, win_size)
    input_list.extend(temp_in)
    output_list.extend(temp_out)

In [42]:
input_data = np.reshape(input_list, (len(input_list), win_size - 1, 1))
print(input_data.shape)

#Normalize data
input_data = input_data / float(len(chars))

(2005467, 99, 1)


c(v)

In [49]:
#One hot spot
from tensorflow import keras
from keras.utils import np_utils
output_data = np_utils.to_categorical(output_list)
print(output_data.shape)

(2005467, 68)


c(vi), c(vii), c(viii), c(ix), c(x)

In [50]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,LSTM
from tensorflow.keras.callbacks import ModelCheckpoint

#Define the LSTM model
LSTM_model = Sequential()
LSTM_model.add(LSTM(256, input_shape = (input_data.shape[1], input_data.shape[2])))
LSTM_model.add(Dropout(0.2))
LSTM_model.add(Dense(output_data.shape[1], activation='softmax'))
print(LSTM_model.summary())

LSTM_model.compile(loss = 'categorical_crossentropy', optimizer = 'adam')

filepath = "../LSTM/weights-improvement-{epoch:02d}-{loss:.2f}-bigger.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor = 'loss', verbose = 1, save_best_only = True, mode = 'min')
callbacks_list = [checkpoint]

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 256)               264192    
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense (Dense)               (None, 68)                17476     
                                                                 
Total params: 281,668
Trainable params: 281,668
Non-trainable params: 0
_________________________________________________________________
None


c(xi)

In [104]:
LSTM_model.fit(input_data, output_data, epochs = 30, batch_size = 128, callbacks = callbacks_list)

Epoch 1/30
15668/15668 [==============================] - ETA: 0s - loss: 2.1199
Epoch 00001: loss improved from 2.22162 to 2.11993, saving model to ../LSTM\weights-improvement-01-2.12-bigger.hdf5
15668/15668 [==============================] - 4062s 259ms/step - loss: 2.1199
Epoch 2/30
15668/15668 [==============================] - ETA: 0s - loss: 2.0526
Epoch 00002: loss improved from 2.11993 to 2.05258, saving model to ../LSTM\weights-improvement-02-2.05-bigger.hdf5
15668/15668 [==============================] - 4211s 269ms/step - loss: 2.0526
Epoch 3/30
15668/15668 [==============================] - ETA: 0s - loss: 2.0013
Epoch 00003: loss improved from 2.05258 to 2.00130, saving model to ../LSTM\weights-improvement-03-2.00-bigger.hdf5
15668/15668 [==============================] - 4282s 273ms/step - loss: 2.0013
Epoch 4/30
15668/15668 [==============================] - ETA: 0s - loss: 1.9614
Epoch 00004: loss improved from 2.00130 to 1.96144, saving model to ../LSTM\weights-improve

In [154]:
init = 'There are those who take mental phenomena naively, just as they would physical phenomena. This school of psychologists tends not to emphasize the object.'

words = [char_dict[c] for c in init[-99:].lower()]

characters = 0
while characters <= 1000:
    seq = np.reshape(words, (1, len(words), 1))
    seq = seq / float(len(chars))
    char_pred = LSTM_model.predict(seq, verbose = 0)
    index = np.argmax(char_pred)
    
    if int_dict[index] != '\n':
        characters += 1
        init += int_dict[index]
    
    words.append(index)
    words = words[1:len(words)]

print(init)

There are those who take mental phenomena naively, just as they would physical phenomena. This school of psychologists tends not to emphasize the object. the to the the sh the tee toe toe toe toe to the to the the th the the ao toe a sh the the tee the toe toe tee th tee to the ae the the the toe the toe aa t ae the tee to the toe the toe toe tne to toe the to the toe to the the toe to the toe an toe the to th th te the 1n th the toe th the toe the toe the soe the toe soe the  and toe th to t an te the te th th the the toe to th the the toe the toe tee toe th tne tee toe toe soe an the oh the toee the teee the tee toee to the th th tee tee toe toe toe toe to th tee toe ao the toe toe toe tee ane tee toe toe to th  ao th the te the toe toe the toe th the th th the the aed ae the toe toe toe toe toe soe toe toe toe an toe toe to the the soe toe the an the tee toe to  - the to the the the a to toe the the to th the the the toe soe to the the to th the to the th the aed to tee te the to th